# Debug MM GAN


In [1]:

import numpy as np
import sys
sys.path.append('/home/tl3/repos/project-rcGAN/jess_version/rcGAN/')

from torch.utils.data import DataLoader
import pytorch_lightning as pl
from typing import Optional
from data.datasets.MM_data import MassMappingDataset_Test, MassMappingDataset_Train, MassMappingDataset_Val


/home/tl3/.conda/envs/cGAN/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from data.lightning.MassMappingDataModule import MMDataTransform
import pathlib
from utils.mri import transforms

/home/tl3/repos/project-rcGAN/jess_version/rcGAN/fastmri/__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [3]:

data_path = '/share/gpu0/tl3/mass_map_dataset/kappa_debug/kappa_run_04974.npy'
kappa = np.load(data_path, allow_pickle=True).astype(np.complex128)


data_dir = '/share/gpu0/tl3/mass_map_dataset/kappa_debug/'


In [4]:
print(kappa.shape)
print(kappa.dtype)

(1024, 1024)
complex128


In [5]:

mm_data_transform = MMDataTransform(args=None)
mm_dataset_train = MassMappingDataset_Train(data_dir=data_dir, transform=mm_data_transform)
new_data = mm_dataset_train.__getitem__(0)


In [7]:
theta=5.0
ng=1024
ngal=30

In [8]:

gamma = MMDataTransform.noise_maker(theta, ng, ngal, kappa)


# gamma = self.gamma_gen(kappa)

# kappa.shape




In [9]:
class MMDataModule(pl.LightningDataModule):
    """
    DataModule used for semantic segmentation in geometric generalization project.
    """
    def __init__(self, args, big_test=False):
        """The 'args' come from the config.yml file. See the docs for further information."""
        super().__init__()
        self.prepare_data_per_node = True
        self.args = args
        self.big_test = big_test

    def prepare_data(self):
        pass

    def setup(self, stage: Optional[str] = None):
        #Assign train/val datasets for use in dataloaders

        train_data = MassMappingDataset_Train(
            data_dir=pathlib.Path('/share/gpu0/tl3/mass_map_dataset') / 'kappa_debug/',
            transform=MMDataTransform(self.args, test=False),
        )

        dev_data = MassMappingDataset_Val(
            data_dir=pathlib.Path('/share/gpu0/tl3/mass_map_dataset') / 'kappa_debug/',
            transform=MMDataTransform(self.args, test=True),
            big_test=self.big_test
        )    

        test_data = MassMappingDataset_Test(
            data_dir=pathlib.Path('/share/gpu0/tl3/mass_map_dataset') / 'kappa_debug/',
            transform=MMDataTransform(self.args, test=True),
            big_test=True
        )

        self.train, self.validate, self.test = train_data, dev_data, test_data


    # define your dataloaders
    # again, here defined for train, validate and test, not for predict as the project is not there yet.
    def train_dataloader(self):
        return DataLoader(
            dataset=self.train,
            batch_size=1,
            num_workers=4,
            drop_last=True,
            pin_memory=False
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.validate,
            batch_size=1,
            num_workers=4,
            drop_last=True,
            pin_memory=False
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=self.test,
            batch_size=4,
            num_workers=4,
            pin_memory=False,
            drop_last=False
        )


In [10]:
args = {
    'batch_size': 1,
    'data_path': '/share/gpu0/tl3/mass_map_dataset'

}


mm_data_module = MMDataModule(args)


In [11]:
mm_data_module.setup()



In [13]:
a = mm_data_module.train_dataloader()
b = mm_data_module.val_dataloader()
c = mm_data_module.test_dataloader()

In [64]:
pt_gamma = transforms.to_tensor(gamma)

In [65]:
pt_gamma.shape

torch.Size([1024, 1024, 2])

In [66]:
pt_gamma = pt_gamma.permute(2, 0, 1) 

In [67]:
normalized_gamma, mean, std = transforms.normalize_instance(pt_gamma)

In [68]:
normalized_gt = transforms.normalize(pt_kappa, mean, std)

In [69]:
mean

tensor(-7.1737e-05, dtype=torch.float64)

In [70]:
std

tensor(0.2309, dtype=torch.float64)